# Chat Bot:
* **This model is based on the End-to-End Memory network. This model is trained using babi dataset, published by Facebook research.**
* The summary of the data is following:
    1. A story of 4-5 sentences is provided to the model.
    2. A question is given to the model based on the story given.
    3. Answer is binary i.e. yes or no type.
* Model summary is as given below:
    1. The story is used as the memory.
    2. The story is embedded using embedding matrix.
    3. Question is matched with the question, which is also embedded with different embedding matrix. **Softmax activation** function is used for scoring the match.
    4. Another embedded matrix is also used in the model, this vectorize the story, which is then added with the match matrix.This is called response
    5. This inturn, is concatnated with the question (which is embedded) to produce the input for decoder(answer).
    6. This is then passed to **LSTM** layer.
    7. And then finally connected to a **Dense** layer.
    8. This is activated with the **sigmoid** fuction to produe probability of each of the word.
    9. Answer will be with highest probability.

In [1]:
import pickle
import numpy as np
import tensorflow as tf

In [2]:
with open('train_qa.txt', 'rb') as fp:
    train_data = pickle.load(fp)

In [3]:
with open('test_qa.txt', 'rb') as fp:
    test_data = pickle.load(fp)

In [4]:
vocab  = set()
all_data = train_data+test_data
len(all_data)

11000

In [5]:
for story,question, answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [6]:
vocab.add('yes')
vocab.add('no')

In [7]:
vocab_len = len(vocab) + 1
story_max_len = max([len(data[0]) for data in all_data])
ques_max_len = max([len(data[1]) for data in all_data])

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
tokenizer = Tokenizer(filters = [])

In [10]:
tokenizer.fit_on_texts(vocab)
word_index = tokenizer.word_index

In [11]:
def vectorized(data,word_index = word_index, max_story = story_max_len, max_ques = ques_max_len ):
    #for the stories(x)
    X = []
    
    #for the question(q)
    Xq = []
    
    #for the answer(a)
    
    A = []
    
    for story, question, answer in data:
        
        #assigning index for every word in story
        x = [word_index[word.lower()] for word in story]
        
        #assigning index for every word in story
        xq = [word_index[word.lower()] for word in question]
        
        # assign index for the answer
        
        a = np.zeros(len(word_index) + 1)
        a[word_index[answer]] =1
        
        X.append(x)
        
        Xq.append(xq)
        
        A.append(a)
        
        
    return (pad_sequences(X, maxlen= max_story), pad_sequences(Xq, maxlen= max_ques), np.array(A))

In [12]:
train_story , train_question , train_answer = vectorized(train_data, word_index)

In [13]:
test_story, test_question, test_answer = vectorized(test_data)

In [14]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Permute, add, concatenate, dot, Activation, Input, Embedding, Dropout

In [15]:
input_seq  = Input((story_max_len,))
ques = Input((ques_max_len,))

Now we can build the encoder as defined in the paper , "End to End Menory Netowrks"

Encoder M, this is used to store and encode memories

In [16]:
encoder_m = Sequential()
encoder_m.add(Embedding(input_dim= vocab_len, output_dim = 64))
encoder_m.add(Dropout(0.2))

Now we can encode C encoder, the output_dimension will be equal to the max length of the question

In [17]:
encoder_c = Sequential()
encoder_c.add(Embedding(input_dim = vocab_len, output_dim = ques_max_len))
encoder_c.add(Dropout(0.2))

Now we can create a question encoder, which will take question as input vector and provide the embedded vector. Output diemensions would be 64 in this case as well

In [18]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim = vocab_len, output_dim= 64,input_length = ques_max_len ))
question_encoder.add(Dropout(0.2))

In [19]:
# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = encoder_m(input_seq)
input_encoded_c = encoder_c(input_seq)
question_encoded = question_encoder(ques)

In [20]:
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

In [21]:
# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

In [22]:
# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

In [23]:
answer

<tf.Tensor 'concatenate/Identity:0' shape=(None, 6, 220) dtype=float32>

In [24]:
answer = LSTM(32)(answer)  # (samples, 32)

In [25]:
# Regularization with Dropout
answer = Dropout(0.5)(answer)
answer = Dense(vocab_len)(answer)  # (samples, vocab_size)

In [26]:
answer = Activation('sigmoid')(answer)

# build the final model
model = Model([input_seq, ques], answer)
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 156)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
sequential (Sequential)         multiple             2432        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 6, 64)        2432        input_2[0][0]                    
______________________________________________________________________________________________

In [28]:
history = model.fit([train_story, train_question], train_answer,batch_size=32,epochs=120,validation_data=([test_story, test_question], test_answer))

Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 9s 901us/sample - loss: 0.1143 - accuracy: 0.9545 - val_loss: 0.0367 - val_accuracy: 0.9737
Epoch 2/120
10000/10000 [==============================] - 2s 237us/sample - loss: 0.0381 - accuracy: 0.9739 - val_loss: 0.0365 - val_accuracy: 0.9738
Epoch 3/120
10000/10000 [==============================] - 2s 237us/sample - loss: 0.0369 - accuracy: 0.9738 - val_loss: 0.0366 - val_accuracy: 0.9735
Epoch 4/120
10000/10000 [==============================] - 2s 237us/sample - loss: 0.0366 - accuracy: 0.9737 - val_loss: 0.0366 - val_accuracy: 0.9738
Epoch 5/120
10000/10000 [==============================] - 2s 240us/sample - loss: 0.0365 - accuracy: 0.9740 - val_loss: 0.0365 - val_accuracy: 0.9738
Epoch 6/120
10000/10000 [==============================] - ETA: 0s - loss: 0.0366 - accuracy: 0.97 - 2s 236us/sample - loss: 0.0365 - accuracy: 0.9735 - val_loss: 0.0365 - val_accuracy: 0.9738
Epo

Epoch 54/120
10000/10000 [==============================] - 2s 225us/sample - loss: 0.0155 - accuracy: 0.9936 - val_loss: 0.0199 - val_accuracy: 0.9911
Epoch 55/120
10000/10000 [==============================] - 2s 227us/sample - loss: 0.0154 - accuracy: 0.9936 - val_loss: 0.0210 - val_accuracy: 0.9907
Epoch 56/120
10000/10000 [==============================] - 2s 228us/sample - loss: 0.0153 - accuracy: 0.9934 - val_loss: 0.0209 - val_accuracy: 0.9909
Epoch 57/120
10000/10000 [==============================] - 2s 227us/sample - loss: 0.0151 - accuracy: 0.9937 - val_loss: 0.0211 - val_accuracy: 0.9915
Epoch 58/120
10000/10000 [==============================] - 2s 229us/sample - loss: 0.0152 - accuracy: 0.9935 - val_loss: 0.0206 - val_accuracy: 0.9914
Epoch 59/120
10000/10000 [==============================] - 2s 234us/sample - loss: 0.0151 - accuracy: 0.9936 - val_loss: 0.0208 - val_accuracy: 0.9906
Epoch 60/120
10000/10000 [==============================] - 2s 226us/sample - loss: 0.01

Epoch 108/120
10000/10000 [==============================] - 2s 231us/sample - loss: 0.0097 - accuracy: 0.9961 - val_loss: 0.0303 - val_accuracy: 0.9907
Epoch 109/120
10000/10000 [==============================] - 2s 230us/sample - loss: 0.0097 - accuracy: 0.9959 - val_loss: 0.0324 - val_accuracy: 0.9903
Epoch 110/120
10000/10000 [==============================] - 2s 230us/sample - loss: 0.0097 - accuracy: 0.9959 - val_loss: 0.0326 - val_accuracy: 0.9901
Epoch 111/120
10000/10000 [==============================] - 2s 230us/sample - loss: 0.0095 - accuracy: 0.9961 - val_loss: 0.0340 - val_accuracy: 0.9907
Epoch 112/120
10000/10000 [==============================] - 2s 231us/sample - loss: 0.0094 - accuracy: 0.9961 - val_loss: 0.0346 - val_accuracy: 0.9908
Epoch 113/120
10000/10000 [==============================] - 2s 231us/sample - loss: 0.0092 - accuracy: 0.9962 - val_loss: 0.0333 - val_accuracy: 0.9904
Epoch 114/120
10000/10000 [==============================] - 2s 230us/sample - los

In [29]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [30]:
model.save('chatbot_akash.h5')

In [31]:
test_pred_res = model.predict([test_story, test_question])

In [32]:
# lets see a test story:

' '.join(test_data[5][0])


'Daniel went back to the kitchen . Mary grabbed the apple there .'

In [33]:
# lets see the question asked:

' '.join(test_data[5][1])

'Is Daniel in the office ?'

In [34]:
# lets see what is the actual answer

test_data[5][2]

'no'

In [35]:
val_max = np.argmax(test_pred_res[5])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key
        
print("Predicted answer is: ", k)
print("Probability of certainty was: ", test_pred_res[5][val_max])

Predicted answer is:  no
Probability of certainty was:  0.99993086


In [ ]:
s